Tutorial link : https://www.youtube.com/watch?v=WWcgHjuKVqA

In [33]:
!pip install tensorflow
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [34]:
import pandas as pd
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create model

Import pretrained base CNN

In [35]:
conv_base = tf.keras.applications.MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)             #since we'll be inputting images of size 224 * 224 * 3
)

In [36]:
# Now we'll simply add the above layers to our model and build upon it.

model = tf.keras.Sequential([
    conv_base, 
    tf.keras.layers.GlobalAveragePooling2D(),               #to basically flatten
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")          #since binary classfication
])

In [37]:
conv_base.trainable = False             #to fix the weights of base model

# Import data, create datasets and preprocess

In [38]:
CATS = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']

cats_images = []
dogs_images = []

for img in glob.glob('images/*.jpg'):
    if any(cat in img for cat in CATS):
        cats_images.append(img)
    else:
        dogs_images.append(img)

In [39]:
#shuffle data
np.random.shuffle(cats_images)
np.random.shuffle(dogs_images)

In [40]:
#divide into train, validation and test sets
train_d, val_d, test_d = np.split(dogs_images, [int(len(dogs_images)*0.7), int(len(dogs_images)*0.8)])
train_c, val_c, test_c = np.split(cats_images, [int(len(cats_images)*0.7), int(len(cats_images)*0.8)])

In [41]:
# Now create dataframes using pandas
# 2 cols, onw with image name, another with label "cat" or "dog"


train_dog_df = pd.DataFrame({'image':train_d, 'label':'dog'})
val_dog_df = pd.DataFrame({'image':val_d, 'label':'dog'})
test_dog_df = pd.DataFrame({'image':test_d, 'label':'dog'})

train_cat_df = pd.DataFrame({'image':train_c, 'label':'cat'})
val_cat_df = pd.DataFrame({'image':val_c, 'label':'cat'})
test_cat_df = pd.DataFrame({'image':test_c, 'label':'cat'})

In [42]:
#now concat thes dataframes, and create trainj, val and test data

train_df = pd.concat([train_dog_df, train_cat_df])
val_df = pd.concat([val_dog_df, val_cat_df])
test_df = pd.concat([test_dog_df, test_cat_df])

In [43]:
BATCH_SIZE = 32
IMG_HEIGHT = IMG_WIDTH = 224

#create the ImageDataGenerator object and rescale the images
train_gen = ImageDataGenerator(rescale = 1./255)
val_gen = ImageDataGenerator(rescale = 1./255)
test_gen = ImageDataGenerator(rescale = 1./255)

#convert them into a dataset
train_dataset = train_gen.flow_from_dataframe(
    dataframe = train_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)

val_dataset = val_gen.flow_from_dataframe(
    dataframe = val_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)

test_dataset = test_gen.flow_from_dataframe(
    dataframe = test_df,
    class_mode = "binary",
    x_col = "image",
    y_col = "label",
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    target_size = (IMG_HEIGHT, IMG_WIDTH)
)


Found 5173 validated image filenames belonging to 2 classes.
Found 739 validated image filenames belonging to 2 classes.
Found 1478 validated image filenames belonging to 2 classes.


# Compile and fit model

In [44]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [45]:
#for PIL error
!pip install pillow


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [46]:
model.fit(
    train_dataset,
    epochs = 5,
    validation_data = (val_dataset)
)

#validation acc starts decreasing after 5th epoch while acc is increasing, ie overfit. 

Epoch 1/5


/home/users/harshit.sahu/Desktop/github_pros/cnn_transfer_learning/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/162 ━━━━━━━━━━━━━━━━━━━━ 132s 788ms/step - accuracy: 0.9479 - loss: 0.1257 - val_accuracy: 0.9878 - val_loss: 0.0282
Epoch 2/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 126s 774ms/step - accuracy: 0.9935 - loss: 0.0169 - val_accuracy: 0.9824 - val_loss: 0.0445
Epoch 3/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 128s 787ms/step - accuracy: 0.9968 - loss: 0.0109 - val_accuracy: 0.9851 - val_loss: 0.0492
Epoch 4/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 145s 892ms/step - accuracy: 0.9984 - loss: 0.0050 - val_accuracy: 0.9838 - val_loss: 0.0399
Epoch 5/5
162/162 ━━━━━━━━━━━━━━━━━━━━ 117s 720ms/step - accuracy: 0.9981 - loss: 0.0066 - val_accuracy: 0.9919 - val_loss: 0.0353


In [47]:
model.evaluate(test_dataset, verbose = 2)

/home/users/harshit.sahu/Desktop/github_pros/cnn_transfer_learning/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


47/47 - 31s - 661ms/step - accuracy: 0.9939 - loss: 0.0178


[0.017750997096300125, 0.9939106702804565]